트레이닝 되지 않은 AutoModel, AutoTokenizer 로 pretrained된 BERT 모델을 로드하고,

코사인유사도를 계산하여 코사인유사도가 가장 높은 답변을 제공합니다.

임베딩은 사전 계산되어 계산 속도를 개선합니다.

In [1]:
import pandas as pd

df = pd.read_csv('public_result.csv')
df.head()

# 질문 리스트
questions = df['question'].tolist()
answers = df['answer'].tolist()

In [2]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel

# BERT 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("beomi/KcBERT-base")
model = AutoModel.from_pretrained("beomi/KcBERT-base")

question_embeddings = torch.load('embeddings/public_question_embeddings.pth', weights_only=True)

# 입력 문장을 임베딩으로 변환
def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)  # 임베딩 벡터 반환

# 입력 질문과 사전 임베딩 간의 코사인 유사도 계산
def find_most_similar_answer(input_question, question_embeddings, answers):
    input_embedding = get_embedding(input_question)  # 입력 질문 임베딩

    max_similarity = -1
    best_answer = None

    # 각 사전 계산된 질문 임베딩과 유사도 비교
    for i, question_embedding in enumerate(question_embeddings):
        similarity = F.cosine_similarity(input_embedding, question_embedding).item()

        if similarity > max_similarity:
            max_similarity = similarity
            best_answer = answers[i]

    return best_answer, max_similarity

# 입력 질문 예시
input_question = "서류제출은 어디에 해?"
answers = df['answer'].tolist()

# 유사도 기반 답변 찾기
answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)
print("유사도:", max_similarity)


c:\Users\user\AppData\Local\Programs\Python\Python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


챗봇 답변: 서류 제출 창구 번호를 확인 중입니다. 잠시만 기다려 주십시오.
유사도: 0.8029338121414185


In [3]:
input_question = "서류제출은 어디?"
answers = df['answer'].tolist()

# 유사도 기반 답변 찾기
answer, max_similarity = find_most_similar_answer(input_question, question_embeddings, answers)
print("챗봇 답변:", answer)
print("유사도:", max_similarity)

챗봇 답변: 기본 증명서, 가족관계증명서, 부모 가족관계증명서, 자녀 가족관계증명서, 주민등록등본, 소명자료를 내셔야 합니다.
유사도: 0.7472295761108398
